In [2]:
import pandas as pd
import seaborn as sns

import requests
from bs4 import BeautifulSoup

import json
import datetime

In [3]:
# Getting each game date with format:

def date(string):
    return string[17:19]+"/"+string[15:17]+"/"+string[11:15]

In [4]:
# Getting each game month in an integer:

def get_month(string):
    return int(string[15:17])

In [5]:
# Getting each game year in an integer:

def get_year(string):
    return int(string[11:15])

In [6]:
# Using datetime library to convert full name month to 

def convert_month(string):

    datetime_object = datetime.datetime.strptime(string, "%B")

    month_number = datetime_object.month

    return month_number

In [7]:
# Seasons starts after september and ends before august:

def convert_season(row):
    if row["month"] >= 8:
        return int(row["season"][:4])
    else:
        return int(row["season"][-4:])

# Apply will be apply to all the rows

#### Importing kaggle dataset:

In [8]:
data = pd.read_csv("../data/games.csv", encoding="ISO-8859-1")

#### Let's investigate this dataset:

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743423 entries, 0 to 743422
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   GAME_ID     743423 non-null  object 
 1   TEAM        743423 non-null  object 
 2   OPPT        743423 non-null  object 
 3   TEAM_SCORE  743423 non-null  int64  
 4   OPPT_SCORE  743423 non-null  int64  
 5   RESULT      743423 non-null  object 
 6   SCORE_DIFF  743423 non-null  int64  
 7   PLAYER      743423 non-null  object 
 8   MP          743423 non-null  float64
 9   FG          743423 non-null  int64  
 10  FGA         743423 non-null  int64  
 11  FG3         743423 non-null  int64  
 12  FG3A        743423 non-null  int64  
 13  FT          743423 non-null  int64  
 14  FTA         743423 non-null  int64  
 15  ORB         743423 non-null  int64  
 16  DRB         743423 non-null  int64  
 17  TRB         743423 non-null  int64  
 18  AST         743423 non-null  int64  
 19  ST

In [10]:
data.columns

Index(['GAME_ID', 'TEAM', 'OPPT', 'TEAM_SCORE', 'OPPT_SCORE', 'RESULT',
       'SCORE_DIFF', 'PLAYER', 'MP', 'FG', 'FGA', 'FG3', 'FG3A', 'FT', 'FTA',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS',
       'PTS', 'TOTAL_MINS', 'STARTER'],
      dtype='object')

In [11]:
data.isnull().sum()

GAME_ID       0
TEAM          0
OPPT          0
TEAM_SCORE    0
OPPT_SCORE    0
RESULT        0
SCORE_DIFF    0
PLAYER        0
MP            0
FG            0
FGA           0
FG3           0
FG3A          0
FT            0
FTA           0
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PLUS_MINUS    0
PTS           0
TOTAL_MINS    0
STARTER       0
dtype: int64

#### This dataset is extremely clean but we will have to manipulate it in order to get some new columns to proceed with our analysis:

In [12]:
data["DATE"] = data["GAME_ID"].apply(date)

In [13]:
data["MONTH"] = data["GAME_ID"].apply(get_month)
data["YEAR"] = data["GAME_ID"].apply(get_year)

In [14]:
data.head(5)

,GAME_ID,TEAM,OPPT,TEAM_SCORE,OPPT_SCORE,RESULT,SCORE_DIFF,PLAYER,MP,FG,...,BLK,TOV,PF,PLUS_MINUS,PTS,TOTAL_MINS,STARTER,DATE,MONTH,YEAR
0,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Jae Crowder,36.683333,6,...,1,1,3,10,16,48,Starter,31/12/2020,12,2020
1,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Mikal Bridges,36.533333,6,...,1,0,0,-2,16,48,Starter,31/12/2020,12,2020
2,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Devin Booker,35.300000,10,...,1,6,3,-2,25,48,Starter,31/12/2020,12,2020
3,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Chris Paul,29.833333,4,...,0,3,2,12,11,48,Starter,31/12/2020,12,2020
4,/boxscores/202012310UTA.html,Phoenix Suns,Utah Jazz,106,95,W,11,Deandre Ayton,28.333333,2,...,2,1,3,-1,4,48,Starter,31/12/2020,12,2020


#### Let's import the json file of the scrapped dataset:

In [15]:
f = open("../data/pom_database.json",)

pom_data = json.load(f)

f.close()

In [16]:
pom = pd.DataFrame(pom_data)

#### Let's first clean the month column:

In [17]:
pom["month"] = pom["pom_month"].apply(convert_month)

#### And then the year column:

In [18]:
pom["year"] = pom.apply(convert_season, axis=1)

#### Web scrapping dataset goes since 1979 and kaggle dataset only since 1996.

#### I will keep information from the years since 1996:

In [19]:
pom = pom[pom["year"] >= 1996]

#### Finally we, cross information from both datasets and keep only games from each player of the month:

In [20]:
pom.head()

,season,player,conference,pom_month,team,position,month,year
0,2020-2021,Russell Westbrook,East,May,Washington Wizards,G,5,2021
1,2020-2021,Stephen Curry,West,May,Golden State Warriors,G,5,2021
2,2020-2021,Julius Randle,East,April,New York Knicks,PF,4,2021
3,2020-2021,Stephen Curry,West,April,Golden State Warriors,G,4,2021
4,2020-2021,James Harden,East,March,Brooklyn Nets,SG,3,2021


In [21]:
for i, pom_player in pom.iterrows():
    print(i)
    
    data.loc[
        (data.PLAYER == pom_player.player) &
        (data.MONTH == pom_player.month) &
        (data.YEAR == pom_player.year),
        "POM"
    ] = True

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265


#### Now let's get a shorter dataframe from each "Player of the month" stats:

In [21]:
data_pom = data[data["POM"] == True]

In [22]:
data_pom.head(10)

,GAME_ID,TEAM,OPPT,TEAM_SCORE,OPPT_SCORE,RESULT,SCORE_DIFF,PLAYER,MP,FG,...,TOV,PF,PLUS_MINUS,PTS,TOTAL_MINS,STARTER,DATE,MONTH,YEAR,POM
8322,/boxscores/202002290MEM.html,Los Angeles Lakers,Memphis Grizzlies,88,105,L,-17,LeBron James,34.316667,8,...,5,1,-12,19,48,Starter,29/02/2020,2,2020,True
8362,/boxscores/202002290BOS.html,Boston Celtics,Houston Rockets,110,111,L,-1,Jayson Tatum,44.566667,9,...,5,2,-5,32,53,Starter,29/02/2020,2,2020,True
8922,/boxscores/202002260UTA.html,Boston Celtics,Utah Jazz,114,103,W,11,Jayson Tatum,38.416667,13,...,2,0,8,33,48,Starter,26/02/2020,2,2020,True
8972,/boxscores/202002250POR.html,Boston Celtics,Portland Trail Blazers,118,106,W,12,Jayson Tatum,34.433333,14,...,2,1,22,36,48,Starter,25/02/2020,2,2020,True
9035,/boxscores/202002250LAL.html,Los Angeles Lakers,New Orleans Pelicans,118,109,W,9,LeBron James,34.200000,17,...,7,1,12,40,48,Starter,25/02/2020,2,2020,True
9423,/boxscores/202002230LAL.html,Boston Celtics,Los Angeles Lakers,112,114,L,-2,Jayson Tatum,36.550000,12,...,2,3,6,41,48,Starter,23/02/2020,2,2020,True
9435,/boxscores/202002230LAL.html,Los Angeles Lakers,Boston Celtics,114,112,W,2,LeBron James,35.483333,9,...,2,4,-1,29,48,Starter,23/02/2020,2,2020,True
9735,/boxscores/202002210LAL.html,Los Angeles Lakers,Memphis Grizzlies,117,105,W,12,LeBron James,35.583333,10,...,1,0,6,32,48,Starter,21/02/2020,2,2020,True
9851,/boxscores/202002210MIN.html,Boston Celtics,Minnesota Timberwolves,127,117,W,10,Jayson Tatum,35.533333,8,...,2,1,0,28,48,Starter,21/02/2020,2,2020,True
10062,/boxscores/202002130BOS.html,Boston Celtics,Los Angeles Clippers,141,133,W,8,Jayson Tatum,47.883333,14,...,4,4,4,39,58,Starter,13/02/2020,2,2020,True


#### Let's export this dataset as .csv:

In [23]:
# Exporting the cleaner dataset

data_pom.to_csv("../data/data_pom.csv", index = False)